本程式想要根據日本的地址來預測該地址的郵遞區號
<br>輸入 : 北海道帯広市西１２条北１丁目８
<br>輸出 : 〒080-0042(實際的輸出只會有數字的部分→0800042)　
<br>

郵遞區號每一個位數都是[0 , 1 , 2 , 3 , 4 , 5 , 6 , 7 , ,8 , 9]中的其中一個數字
<br>若用傳統的多分類問題的手法來分析這個case，那出輸出的可能性將呈現指數級增長，我們不太可能直接這樣幹
<br>所以需要對每一個位數分別預測是屬於0~9中的哪一個class
而這個case即屬於Multi-Label的問題

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import tensorflow as tf
import math
from data_augmentation import data_augmentation_function_set

In [2]:
vocabulary = []
f = open('addresses4.txt' , 'rb')
data = f.readlines()[:100000] # 取前100000筆來訓練
address_ = []
postal_ = [] 
for line in data:
    postal , address = line.decode('utf8').strip().split('　')
    address_.append(address)
    postal_split = postal[1:].split('-')[0] + postal[1:].split('-')[1]
    postal_temp = []
    for i in postal_split:
        postal_temp.append(int(i)) # 每個postal都只有 7 個數字
    postal_.append(postal_temp)
    for element in list(postal):
        vocabulary.append(element)
    for element in list(address):
        vocabulary.append(element)

vocabulary = sorted(list(set(vocabulary)))

In [3]:
# 執行data augmentation，防止overfitting
function_set = data_augmentation_function_set()
address_augmentation = []
for address in address_:
    # data augmentation總共有4種模式，這裡針對每筆資料隨機選1種
    dart = np.random.choice(a = 4 ,
                            size = 1 ,
                            p = [0.25 , 0.25 , 0.25 , 0.25]) 
    dart = dart[0]
    address_augmentation.append(function_set[dart](address))

address_ = address_ + address_augmentation
postal_ = postal_ * 2

In [4]:
word_to_int = {}
for i , word in enumerate(vocabulary):
    word_to_int[word] = i

word_to_int['<PAD>'] = i + 1 # 將'<PAD>'也列入詞庫   
word_to_int['<UNK>'] = i + 2 # augmentation後address會出現很多沒出現過的字，這些字都視作'<UNK>'

int_to_word = dict(zip(word_to_int.values() , word_to_int.keys()))   

max_len = 0 # 取得地址最大長度
for sentence in address_:
    max_len = max(max_len , len(sentence))

In [5]:
address_encode = []
for sentence in address_:
    temp = []
    for word in sentence:
        if word in word_to_int.keys():
            temp.append(word_to_int[word])
        else:
            temp.append(word_to_int['<UNK>'])
    while len(temp) < max_len:
        temp.append(word_to_int['<PAD>'])
    address_encode.append(temp)    

    
address_encode = np.array(address_encode)
postal_ = np.array(postal_)

# 因為地址是照地區排序的，若沒有做shuffle這個動作，有些地區的地址會在training的過程中完全沒見過，導致最後testing時無法預測
indices = np.arange(len(address_encode))
np.random.shuffle(indices)
address_encode = address_encode[indices]
postal_ = postal_[indices]

split_at = len(address_) - len(address_) // 10
address_train , address_val = address_encode[:split_at] , address_encode[split_at:]
postal_train , postal_val = postal_[:split_at] , postal_[split_at:]

In [6]:
# 超參數
rnn_size = 256
batch_size = 300
vocab_size = len(word_to_int)
embedding_size = 200

In [7]:
sources = tf.placeholder(tf.int32 , [None , max_len] , name = 'sources')
targets = tf.placeholder(tf.int64 , [None , 7] , name = 'targets')
targets_onehot = tf.one_hot(tf.reshape(targets , [-1]) , depth = 10)

embeddings = tf.Variable(tf.random_uniform([vocab_size , embedding_size], -1, 1))
embed_input = tf.nn.embedding_lookup(embeddings , sources)

def lstm_cell(hidden_size): 
    cell = tf.contrib.rnn.LSTMCell(hidden_size)
    return cell


with tf.variable_scope('LSTM'):
    mlstm_cell = tf.contrib.rnn.MultiRNNCell([lstm_cell(rnn_size) for _ in range(1)] , 
                                              state_is_tuple = True)
    
    outputs , states = tf.nn.dynamic_rnn(mlstm_cell , 
                                         inputs = embed_input ,
                                         dtype = tf.float32) # outputs → [batch_size , max_len , rnn_size]


outputs_ = tf.reshape(outputs , [-1 , max_len * rnn_size]) # [batch_size , max_len * rnn_size]
# 在最後一層加上一個全連接層
weights = tf.Variable(tf.truncated_normal([max_len * rnn_size , 7 * 10] , mean = 0.01 , stddev = 0.1))
biases = tf.Variable(tf.zeros([1 , 7 * 10]) + 0.01)
logits = tf.matmul(outputs_ , weights) + biases  # [batch_size , 7 * 10]
logits = tf.reshape(logits , [-1 , 10]) # [batch_size * 7 , 10]

In [8]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits , labels = targets_onehot))
train_op = tf.train.AdamOptimizer(0.001).minimize(cost)

'''
prediction:[1 2 3 5 5 6 7      targets:[1 2 3 4 5 6 7
            2 3 4 5 6 6 8]              2 3 4 5 6 7 8]

tf.equal(prediction , targets) → [True True True False True True True    → correct = [1 1 1 0 1 1 1
                                   True True True True  True True True]                 1 1 1 1 1 1 1]

tf.reduce_min(correct , axis = 1) → [0
                                      1]

只要prediction中1個數字錯，就相當於全錯
'''
prediction = tf.reshape(logits , [-1 , 7 , 10]) # prediction → [batch_size , 7 , 10]
prediction = tf.argmax(prediction , axis = 2)   # prediction → [batch_size , 7]
correct = tf.cast(tf.equal(prediction , targets) , tf.float32)
accuracy = tf.reduce_mean(tf.reduce_min(correct , axis = 1))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [9]:
# 決定每個batchy訓練資料的index
step = (math.ceil(len(address_train) / batch_size)) * batch_size
temp = []
j = 0
index = []
for ii in range(0 , step):
    j = j + 1
    if j > len(address_train):
        j = j - (len(address_train))   
    temp.append(j)  
    if len(temp) == batch_size:
        index.append(temp)
        temp = []

index = list(np.array(index) - 1)

In [10]:
sess = tf.Session() 
sess.run(tf.global_variables_initializer())


batch_loss_record = []
batch_accuracy_record = []
batch_loss_mean_record = []
batch_accuracy_mean_record = []
val_loss_record = []
val_accuracy_record = []
for epoch_i in range(0 , 3):        
    for batch_i in range(0 , len(index)):        
        batch_source = address_train[index[batch_i] , :]
        batch_target = postal_train[index[batch_i] , :]
        
        _ , batch_loss , batch_accuracy , prediction_result =\
        sess.run([train_op , cost , accuracy , prediction] , 
                 feed_dict = {sources : batch_source , 
                              targets : batch_target})
        
        batch_loss_record.append(batch_loss)
        batch_accuracy_record.append(batch_accuracy)        
        if len(batch_loss_record) == 20:
            batch_loss_mean = np.array(batch_loss_record).mean()
            batch_accuracy_mean = np.array(batch_accuracy_record).mean()    
            batch_loss_mean_record.append(batch_loss_mean) 
            batch_accuracy_mean_record.append(batch_accuracy_mean)
            batch_loss_record = []
            batch_accuracy_record = []
        
        print('\nepoch : {} , batch : {} , loss : {:.2f}'.format(epoch_i , batch_i , batch_loss))
        print('epoch : {} , batch : {} , accuracy : {:.2%}'.format(epoch_i , batch_i , batch_accuracy))
        
        data_index = np.random.randint(batch_size)  
        
        source_visualization = []
        for i in batch_source[data_index]:
            if int_to_word[i] == '<PAD>': break
            source_visualization.append(int_to_word[i])
        source_visualization = ' '.join(source_visualization)
        
        target_visualization = []
        for i in batch_target[data_index]:
            target_visualization.append(str(i))
        target_visualization = ' '.join(target_visualization)

        predict_visualization = []
        for i in prediction_result[data_index]:
            predict_visualization.append(str(i))
        predict_visualization = ' '.join(predict_visualization)
        
        print('Source : {}'.format(source_visualization))
        print('Target : {}'.format(target_visualization))
        print('Predict : {}\n'.format(predict_visualization))
        
        if batch_i % 20 == 0 and batch_i > 0:
            batch_val_source = address_val
            batch_val_target = postal_val
            val_loss , val_accuracy = sess.run([cost , accuracy] , 
                                               feed_dict = {sources : batch_val_source , 
                                                            targets : batch_val_target})
            
            val_loss_record.append(val_loss) 
            val_accuracy_record.append(val_accuracy)                                                   
            
            print('\n' + '=' * 30)
            print('epoch : {} , val loss : {:.2f}'.format(epoch_i , val_loss))
            print('epoch : {} , val accuracy : {:.2%}'.format(epoch_i , val_accuracy))
            print('=' * 30 + '\n')


epoch : 0 , batch : 0 , loss : 3.04
epoch : 0 , batch : 0 , accuracy : 0.00%
Source : 奈 良 県 <UNK> 奈 良 市 登 大 路 町 ５ ９
Target : 6 3 0 8 2 1 3
Predict : 5 9 9 7 8 4 8


epoch : 0 , batch : 1 , loss : 2.21
epoch : 0 , batch : 1 , accuracy : 0.00%
Source : 長 崎 県 佐 世 保 市 島 瀬 町 <UNK> ２ − ２
Target : 8 5 7 0 8 0 6
Predict : 7 3 0 0 0 3 2


epoch : 0 , batch : 2 , loss : 2.21
epoch : 0 , batch : 2 , accuracy : 0.00%
Source : 大 阪 府 枚 方 市 南 楠 葉 １ 丁 目 １ ０ − ４
Target : 5 7 3 1 1 0 5
Predict : 9 5 0 0 0 0 3


epoch : 0 , batch : 3 , loss : 2.08
epoch : 0 , batch : 3 , accuracy : 0.00%
Source : 鹿 児 島 県 鹿 児 島 市 荒 田 ２ 丁 目 ６ ３ − １
Target : 8 9 0 0 0 5 4
Predict : 3 5 9 0 0 5 3


epoch : 0 , batch : 4 , loss : 1.88
epoch : 0 , batch : 4 , accuracy : 0.00%
Source : 愛 媛 県 今 治 市 蒼 社 町 １ 丁 目 ２ − ２ ６
Target : 7 9 4 0 0 5 8
Predict : 1 5 6 0 0 3 4


epoch : 0 , batch : 5 , loss : 1.78
epoch : 0 , batch : 5 , accuracy : 0.00%
Source : 和 <UNK> 山 県 岩 出 市 川 尻 ２ ７ ５ − ３
Target : 6 4 9 6 2 3 1
Predict : 9 6 2 0 1 3 2


epoch : 0 , batch : 47 , loss : 0.76
epoch : 0 , batch : 47 , accuracy : 21.67%
Source : 栃 木 県 市 野 佐 相 生 町 １ ９ ９
Target : 3 2 7 0 0 2 3
Predict : 3 2 7 0 1 0 3


epoch : 0 , batch : 48 , loss : 0.76
epoch : 0 , batch : 48 , accuracy : 18.67%
Source : 佐 賀 県 佐 賀 市 鍋 島 ２ 丁 目 ５ − ２ ７ − １ Ｆ
Target : 8 4 9 0 9 3 7
Predict : 8 4 9 0 9 3 7


epoch : 0 , batch : 49 , loss : 0.76
epoch : 0 , batch : 49 , accuracy : 23.33%
Source : 鹿 児 島 県 霧 島 市 国 分 中 央 ３ 丁 目 ３ ９ − ２ ７
Target : 8 9 9 4 3 3 2
Predict : 8 9 9 4 3 3 2


epoch : 0 , batch : 50 , loss : 0.74
epoch : 0 , batch : 50 , accuracy : 25.67%
Source : 山 口 県 山 口 市 下 小 鯖 ２ ４ ７ ０
Target : 7 5 3 0 2 1 2
Predict : 7 5 3 0 0 0 1


epoch : 0 , batch : 51 , loss : 0.74
epoch : 0 , batch : 51 , accuracy : 20.33%
Source : 大 阪 府 大 阪 市 住 吉 区 遠 里 小 野 １ 丁 <UNK> １ ２
Target : 5 5 8 0 0 3 2
Predict : 5 5 8 0 0 0 2


epoch : 0 , batch : 52 , loss : 0.74
epoch : 0 , batch : 52 , accuracy : 24.00%
Source : 奈 良 県 奈 良 路 登 大 市 町 ３ ０
Target : 6 3 0 8 2 1 3
Predict :


epoch : 0 , batch : 94 , loss : 0.47
epoch : 0 , batch : 94 , accuracy : 45.00%
Source : 山 口 県 山 口 市 桜 畠 ４ 丁 目 ３ − ２ ７
Target : 7 5 3 0 0 2 1
Predict : 7 5 4 0 0 1 1


epoch : 0 , batch : 95 , loss : 0.45
epoch : 0 , batch : 95 , accuracy : 41.33%
Source : 熊 本 県 球 磨 郡 多 良 木 町 大 字 多 良 木 １ ６ ０ ８ − １
Target : 8 6 8 0 5 0 1
Predict : 8 6 8 0 5 0 1


epoch : 0 , batch : 96 , loss : 0.49
epoch : 0 , batch : 96 , accuracy : 44.00%
Source : 東 京 都 新 宿 区 市 谷 田 町 １ 丁 目 １ ９
Target : 1 6 2 0 8 4 3
Predict : 1 6 2 0 8 4 3


epoch : 0 , batch : 97 , loss : 0.51
epoch : 0 , batch : 97 , accuracy : 34.67%
Source : 山 形 県 山 形 市 蔵 王 松 ケ 丘 １ 丁 目 ６ ５ ６ − ２
Target : 9 9 0 2 3 3 8
Predict : 9 9 0 2 3 3 1


epoch : 0 , batch : 98 , loss : 0.48
epoch : 0 , batch : 98 , accuracy : 39.33%
Source : 大 阪 府 大 東 市 新 田 北 町 １ − １ ４
Target : 5 7 4 0 0 5 2
Predict : 5 7 4 0 0 5 6


epoch : 0 , batch : 99 , loss : 0.46
epoch : 0 , batch : 99 , accuracy : 44.00%
Source : 石 川 県 鳳 珠 郡 穴 水 字 此 木 １ ６ − ４ ６
Target : 9 2 7 0 0 5


epoch : 0 , batch : 140 , loss : 0.30
epoch : 0 , batch : 140 , accuracy : 61.67%
Source : 県 木 栃 日 光 市 野 門 ２ ２ ５
Target : 3 2 1 2 7 1 6
Predict : 3 2 1 2 1 3 2


epoch : 0 , val loss : 0.32
epoch : 0 , val accuracy : 58.15%


epoch : 0 , batch : 141 , loss : 0.34
epoch : 0 , batch : 141 , accuracy : 55.33%
Source : 鹿 霧 島 県 児 島 市 国 分 中 央 ３ 丁 目 ４ １ − ２ ３
Target : 8 9 9 4 3 3 2
Predict : 8 9 9 4 3 3 2


epoch : 0 , batch : 142 , loss : 0.33
epoch : 0 , batch : 142 , accuracy : 57.00%
Source : 宮 城 県 登 米 市 迫 町 佐 沼 字 下 田 中 <UNK> ２
Target : 9 8 7 0 5 1 1
Predict : 9 8 7 0 5 1 1


epoch : 0 , batch : 143 , loss : 0.33
epoch : 0 , batch : 143 , accuracy : 55.00%
Source : 大 阪 府 枚 方 市 <UNK> 楠 葉 １ 丁 目 ６ − ８
Target : 5 7 3 1 1 0 5
Predict : 5 7 3 1 1 0 5


epoch : 0 , batch : 144 , loss : 0.34
epoch : 0 , batch : 144 , accuracy : 53.67%
Source : 福 岡 県 宗 像 市 田 島 １ １ ５
Target : 8 1 1 3 5 0 5
Predict : 8 1 1 0 8 4 3


epoch : 0 , batch : 145 , loss : 0.31
epoch : 0 , batch : 145 , accuracy : 62.67%
S


epoch : 0 , batch : 186 , loss : 0.25
epoch : 0 , batch : 186 , accuracy : 69.33%
Source : 愛 知 県 名 古 屋 市 昭 和 区 小 坂 町 ２ 丁 目 ３
Target : 4 6 6 0 0 2 1
Predict : 4 6 6 0 8 0 5


epoch : 0 , batch : 187 , loss : 0.23
epoch : 0 , batch : 187 , accuracy : 69.67%
Source : 富 山 県 射 水 市 <UNK> 町 ２ − ２ ５
Target : 9 3 4 0 0 1 3
Predict : 9 3 4 0 0 0 3


epoch : 0 , batch : 188 , loss : 0.26
epoch : 0 , batch : 188 , accuracy : 65.00%
Source : 岐 阜 県 岐 阜 市 西 <UNK> ６ 丁 目 ２ ３ − １ ０ ７
Target : 5 0 0 8 2 8 6
Predict : 5 0 0 8 8 8 6


epoch : 0 , batch : 189 , loss : 0.24
epoch : 0 , batch : 189 , accuracy : 68.67%
Source : 長 崎 県 佐 世 保 市 黒 <UNK> 町 ４ ９ − ５
Target : 8 5 7 1 1 5 2
Predict : 8 5 7 1 8 5 6


epoch : 0 , batch : 190 , loss : 0.23
epoch : 0 , batch : 190 , accuracy : 65.33%
Source : 静 岡 県 沼 津 市 西 条 町 ３ ４
Target : 4 1 0 0 8 0 4
Predict : 4 1 0 0 8 0 5


epoch : 0 , batch : 191 , loss : 0.22
epoch : 0 , batch : 191 , accuracy : 69.00%
Source : 福 岡 県 直 方 市 大 ６ 感 田 ４ 字 − １
Target : 8 2 2 0 0 0 1
Pre

...


epoch : 2 , batch : 373 , loss : 0.02
epoch : 2 , batch : 373 , accuracy : 97.33%
Source : 兵 庫 県 神 戸 市 長 田 区 駒 ケ <UNK> 南 町 １ − ６ ０
Target : 6 5 3 0 0 4 5
Predict : 6 5 3 0 0 4 5


epoch : 2 , batch : 374 , loss : 0.01
epoch : 2 , batch : 374 , accuracy : 97.00%
Source : 栃 木 県 足 利 市 久 松 <UNK> 町 ５ ７ − １
Target : 3 2 6 0 0 3 4
Predict : 3 2 6 0 0 3 4


epoch : 2 , batch : 375 , loss : 0.01
epoch : 2 , batch : 375 , accuracy : 99.33%
Source : 愛 知 県 豊 橋 市 西 橋 良 町 ９
Target : 4 4 1 8 0 3 9
Predict : 4 4 1 8 0 3 9


epoch : 2 , batch : 376 , loss : 0.01
epoch : 2 , batch : 376 , accuracy : 98.00%
Source : 鹿 児 島 県 霧 島 市 霧 島 田 口 ２ ５ １ ８ − １ ２
Target : 8 9 9 4 2 0 1
Predict : 8 9 9 4 2 0 1


epoch : 2 , batch : 377 , loss : 0.01
epoch : 2 , batch : 377 , accuracy : 97.67%
Source : 福 岡 県 遠 賀 郡 遠 賀 町 大 字 広 渡
Target : 8 1 1 4 3 0 2
Predict : 8 1 1 4 3 0 2


epoch : 2 , batch : 378 , loss : 0.02
epoch : 2 , batch : 378 , accuracy : 96.33%
Source : <UNK> 庫 県 明 石 市 大 観 町 １ １ − ８
Target : 6 7 3 0 8 9 7


epoch : 2 , batch : 419 , loss : 0.03
epoch : 2 , batch : 419 , accuracy : 96.00%
Source : 東 <UNK> 京 都 新 宿 区 市 谷 田 町 ３ 丁 目 １ ９
Target : 1 6 2 0 8 4 3
Predict : 1 6 2 0 8 4 3


epoch : 2 , batch : 420 , loss : 0.02
epoch : 2 , batch : 420 , accuracy : 97.00%
Source : 東 京 都 足 立 区 東 和 ４ 丁 目 ５ − ８
Target : 1 2 0 0 0 0 3
Predict : 1 2 0 0 0 0 3


epoch : 2 , val loss : 0.02
epoch : 2 , val accuracy : 96.78%


epoch : 2 , batch : 421 , loss : 0.01
epoch : 2 , batch : 421 , accuracy : 97.33%
Source : 石 川 県 鳳 珠 <UNK> 郡 穴 水 町 字 此 木 １ − １ ０ ２
Target : 9 2 7 0 0 5 3
Predict : 9 2 7 0 0 5 3


epoch : 2 , batch : 422 , loss : 0.02
epoch : 2 , batch : 422 , accuracy : 96.67%
Source : 石 川 県 鳳 珠 郡 穴 水 町 字 平 野 ３ − ２ １ − ３
Target : 9 2 7 0 0 5 2
Predict : 9 2 7 0 0 5 2


epoch : 2 , batch : 423 , loss : 0.02
epoch : 2 , batch : 423 , accuracy : 97.33%
Source : 宮 城 県 登 米 市 南 方 町 八 の 森 ３ ５ − ４
Target : 9 8 7 0 4 0 1
Predict : 9 8 7 0 4 0 1


epoch : 2 , batch : 424 , loss : 0.01
epoch : 2 , batch : 424 ,


epoch : 2 , batch : 464 , loss : 0.01
epoch : 2 , batch : 464 , accuracy : 99.00%
Source : 縄 県 那 覇 市 字 安 里 ３ ８ ８
Target : 9 0 2 0 0 6 7
Predict : 9 0 2 0 0 6 7


epoch : 2 , batch : 465 , loss : 0.02
epoch : 2 , batch : 465 , accuracy : 96.67%
Source : 福 井 県 坂 井 市 三 国 町 宿 ３ − １ １ − ３ <UNK> ４
Target : 9 1 3 0 0 5 6
Predict : 9 1 3 0 0 5 6


epoch : 2 , batch : 466 , loss : 0.01
epoch : 2 , batch : 466 , accuracy : 98.33%
Source : 鹿 児 島 県 霧 島 市 国 分 野 口 東 １ ０ ５ ８ − ２
Target : 8 9 9 4 3 4 1
Predict : 8 9 9 4 3 4 1


epoch : 2 , batch : 467 , loss : 0.02
epoch : 2 , batch : 467 , accuracy : 96.00%
Source : 山 形 − 寒 河 江 市 内 ノ 袋 １ 丁 目 ６ 県 ４
Target : 9 9 1 0 0 5 7
Predict : 9 9 1 0 0 5 7


epoch : 2 , batch : 468 , loss : 0.01
epoch : 2 , batch : 468 , accuracy : 98.00%
Source : <UNK> 大 阪 府 大 東 市 曙 町 ６ − ２ ６
Target : 5 7 4 0 0 7 6
Predict : 5 7 4 0 0 7 6


epoch : 2 , batch : 469 , loss : 0.01
epoch : 2 , batch : 469 , accuracy : 98.00%
Source : 東 京 都 新 宿 <UNK> 区 市 谷 田 町 ２ 丁 目 ７ − ３ ０ １
Target


epoch : 2 , batch : 510 , loss : 0.02
epoch : 2 , batch : 510 , accuracy : 97.00%
Source : 兵 庫 県 <UNK> 神 戸 市 長 田 区 苅 藻 島 町 ３ 丁 目 ５ − ２ ４
Target : 6 5 3 0 0 3 3
Predict : 6 5 3 0 0 3 3


epoch : 2 , batch : 511 , loss : 0.01
epoch : 2 , batch : 511 , accuracy : 97.33%
Source : 兵 庫 県 神 戸 市 長 田 区 若 松 町 ２ 丁 目 １ − ２ ７ − ２ ０ １
Target : 6 5 3 0 0 3 8
Predict : 6 5 3 0 0 3 8


epoch : 2 , batch : 512 , loss : 0.01
epoch : 2 , batch : 512 , accuracy : 96.67%
Source : 佐 賀 県 多 久 市 東 多 久 町 大 字 納 所 ４ ７ ９ ９
Target : 8 4 6 0 0 1 4
Predict : 8 4 6 0 0 1 4


epoch : 2 , batch : 513 , loss : 0.01
epoch : 2 , batch : 513 , accuracy : 98.67%
Source : 富 山 県 射 水 市 中 老 田 新 ６ ９ ５
Target : 9 3 9 0 3 1 2
Predict : 9 3 9 0 3 1 2


epoch : 2 , batch : 514 , loss : 0.01
epoch : 2 , batch : 514 , accuracy : 97.00%
Source : 東 京 都 墨 田 区 押 上 丁 目 １ ９ − １
Target : 1 3 1 0 0 4 5
Predict : 1 3 1 0 0 4 5


epoch : 2 , batch : 515 , loss : 0.01
epoch : 2 , batch : 515 , accuracy : 98.00%
Source : 大 阪 府 <UNK> 屋 川 市 本 町 ６ − 


epoch : 2 , batch : 557 , loss : 0.01
epoch : 2 , batch : 557 , accuracy : 97.67%
Source : 長 崎 県 佐 <UNK> 世 保 市 小 佐 世 保 町 ２ ５ − ７
Target : 8 5 7 0 8 1 3
Predict : 8 5 7 0 8 1 3


epoch : 2 , batch : 558 , loss : 0.01
epoch : 2 , batch : 558 , accuracy : 98.33%
Source : 埼 玉 県 草 加 市 稲 荷 ５ 丁 目 ５ − １ ６
Target : 3 4 0 0 0 0 3
Predict : 3 4 0 0 0 0 3


epoch : 2 , batch : 559 , loss : 0.01
epoch : 2 , batch : 559 , accuracy : 98.33%
Source : 香 川 県 木 田 郡 三 木 町 大 字 田 中 ４ １ ８ ２
Target : 7 6 1 0 6 1 1
Predict : 7 6 1 0 6 1 1


epoch : 2 , batch : 560 , loss : 0.02
epoch : 2 , batch : 560 , accuracy : 97.67%
Source : 大 阪 府 大 阪 市 住 吉 区 万 代 ２ 丁 目 <UNK> − １ ８
Target : 5 5 8 0 0 5 5
Predict : 5 5 8 0 0 5 5


epoch : 2 , val loss : 0.02
epoch : 2 , val accuracy : 97.41%


epoch : 2 , batch : 561 , loss : 0.02
epoch : 2 , batch : 561 , accuracy : 97.33%
Source : 大 阪 府 大 阪 市 住 吉 区 上 住 吉 ２ 丁 目 １ ２ − ５
Target : 5 5 8 0 0 4 6
Predict : 5 5 8 0 0 4 6


epoch : 2 , batch : 562 , loss : 0.01
epoch : 2 , batch